# COGS 108 - Data Checkpoint

# Names

- Royce Casiano
- Keionni Thompson
- Hargun Singh Mahindra
- Ryan Kim


<a id='research_question'></a>
# Research Question

What criteria should be met in order for an up and coming artist to make it in the music industry? More specifically what musical style elements result in the most Spotify streams for artists? 

# Dataset(s)

## *Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: 2000song.csv
- Link to the dataset: https://www.kaggle.com/iamsumat/spotify-top-2000s-mega-dataset
- Number of observations: 2000

In this dataset Spotify's 2000 most streamed songs are collected and compiled. Along with the name and artist of each song are information for other descriptive variables of the songs.

- Dataset Name: 116ksong.csv
- Link to the dataset: https://www.kaggle.com/tomigelo/spotify-audio-features
- Number of observations: 130663 

In this dataset 130 thousand of Spotify's random featured songs have been pulled from their service and recorded. Similarly to the above datase the name and artist of each song are information for other descriptive variables of the songs. With this set we look to compare the attributes of those in the top 2000 to those which are not.




# Setup

In [51]:
# import pandas library
import pandas as pd

# import numpy library
import numpy as np

# import matplotlib
import matplotlib.pyplot as plt

# Import seaborn - will change style later
#import seaborn as sns


# Statmodels & patsy
#from patsy import dmatrices
#import statsmodels.api as sm

#import scipy.stats as stats

# Data Cleaning

 Describe your data cleaning steps here.

In [52]:
# Here we ar first reading in our data.
hundf = pd.read_csv("data/2000song.csv")
moredf = pd.read_csv('data/116ksong.csv')

In our first data set there are 2000 observations representing the 2000 most streamed songs on Spotify. 
The variables we want to focus on are title,artist, bpm, energy, loudness(.dB), length(ms),acousticness,song duration and speechiness, which were calculated and recorded directly by the spotify platform/API.

Since our first data set represents the top 2000 streamed songs on spotify. From our second dataset we will be focused on songs which are not considered part of the top 2000 to make a comparison. 



In [53]:
#Quick look at what is inside dataset 1
hundf.head(3)


,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,1,Sunrise,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201,94,3,71
1,2,Black Night,Deep Purple,album rock,2000,135,79,50,-11,17,81,207,17,7,39
2,3,Clint Eastwood,Gorillaz,alternative hip hop,2001,168,69,66,-9,7,52,341,2,17,69


In [54]:
#Quick look at what is inside dataset 2
moredf.head(3)


,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.00582,0.743,238373,0.339,0.0,1,0.0812,-7.678,1,0.4090,203.927,4,0.118,15
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.02440,0.846,214800,0.557,0.0,8,0.2860,-7.259,1,0.4570,159.009,4,0.371,0
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.02500,0.603,138913,0.723,0.0,9,0.0824,-5.890,0,0.0454,114.966,4,0.382,56


To work with our data sets we need to create a uniform for them. First we will clean up the column names and isolate for the variables we are interested in. 

In [55]:
#This reordered our variables and eliminated characteristics we are not interested in.
hundf = hundf[['Title','Artist','Beats Per Minute (BPM)','Energy','Loudness (dB)','Liveness','Valence','Length (Duration)','Acousticness','Speechiness']]
moredf = moredf[['track_name','artist_name','tempo','energy','loudness','liveness','valence','duration_ms','acousticness','speechiness']]

hundf.rename(columns={'Title': 'song_name','Artist': 'song_artist','Beats Per Minute (BPM)':'BPM','Energy':'energy','Loudness (dB)':'loudness','Liveness':'liveness','Valence':'valence', 'Length (Duration)':'song_duration','Acousticness':'acousticness','Speechiness':'speechiness'}, inplace=True)
moredf.rename(columns={'track_name': 'song_name','artist_name': 'song_artist','tempo':'BPM','duration_ms':'song_duration'}, inplace=True)




In [56]:
hundf.head(3)

,song_name,song_artist,BPM,energy,loudness,liveness,valence,song_duration,acousticness,speechiness
0,Sunrise,Norah Jones,157,30,-14,11,68,201,94,3
1,Black Night,Deep Purple,135,79,-11,17,81,207,17,7
2,Clint Eastwood,Gorillaz,168,69,-9,7,52,341,2,17


In [57]:
moredf.head(3)

,song_name,song_artist,BPM,energy,loudness,liveness,valence,song_duration,acousticness,speechiness
0,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",YG,203.927,0.339,-7.678,0.0812,0.118,238373,0.00582,0.4090
1,BAND DRUM (feat. A$AP Rocky),YG,159.009,0.557,-7.259,0.2860,0.371,214800,0.02440,0.4570
2,Radio Silence,R3HAB,114.966,0.723,-5.890,0.0824,0.382,138913,0.02500,0.0454


Both data sets were made using the same Spotify API thus the values of each column are calculated in a standard way across the data sets. However the values in some columns were manipulated to better serve the purpose of their original creator.
Next, in order to make the datasets for uniformed we will look for one song that is contained in both datasets then adjust values accordingly to uniform both sets.

In [58]:
for i in range(len(hundf.song_name)):
    if 'bad guy' == hundf.song_name[i]:
        indx = i
hundf.iloc[indx]

song_name              bad guy
song_artist      Billie Eilish
BPM                        135
energy                      43
loudness                   -11
liveness                    10
valence                     56
song_duration              194
acousticness                33
speechiness                 38
Name: 786, dtype: object

In [59]:
for i in range(len(moredf.song_name)):
    if 'bad guy' == moredf.song_name[i]:
        indx = i
moredf.iloc[indx]

song_name              bad guy
song_artist      Billie Eilish
BPM                    131.926
energy                   0.418
loudness               -10.998
liveness                   0.1
valence                  0.578
song_duration           194088
acousticness             0.308
speechiness              0.368
Name: 99368, dtype: object

In [60]:
# Dataset one will be set to match values of dataset two witht the following functions.This is because it will be easier to visualize.
def standardize_BPM(val):
    try:
        output = val * .97722963 
       
    except:
        output = np.nan
        
    return output

def standardize_loud(val):
    try:
        output = val / 1.0001851
       
    except:
        output = np.nan
        
    return output
# Dataset two will be set to match values of dataset one with the following functions. This is because it will be easier to visualize.
def standardize_energy(val):
    try:
        output = val * 102.8708134
  
    except:
        output = np.nan
        
    return output
def standardize_live(val):
    try:
        output = val * 100
    
    except:
        output = np.nan
        
    return output
def standardize_valence(val):
    try:
        output = (val * 100)- 1.8
       
    except:
        output = np.nan
        
    return output 

def standardize_dur(val):
    try:
        output = val / 1000.4536
       
    except:
        output = np.nan
        
    return output  

def standardize_acou(val):
    try:
        output = val * 107.142857
      
    except:
        output = np.nan
        
    return output 

def standardize_speech(val):
    try:
        output = val * 103.26086
      
    except:
        output = np.nan
        
    return output 
# Now we will run the functions
hundf['BPM'] = hundf['BPM'].apply(standardize_BPM)
hundf['loudness'] = hundf['loudness'].apply(standardize_loud)
moredf['energy'] = moredf['energy'].apply(standardize_energy)
moredf['liveness'] = moredf['liveness'].apply(standardize_live)
moredf['valence'] = moredf['valence'].apply(standardize_valence)
moredf['song_duration'] = moredf['song_duration'].apply(standardize_dur)
moredf['acousticness'] = moredf['acousticness'].apply(standardize_acou)
moredf['speechiness'] = moredf['speechiness'].apply(standardize_speech)

To make sure our functions worked correctly let us revisit our chosen song and compare the reworked values.

In [61]:
for i in range(len(moredf.song_name)):
    if 'bad guy' == moredf.song_name[i]:
        indx = i
Billie2 = moredf.iloc[indx]
for i in range(len(hundf.song_name)):
    if 'bad guy' == hundf.song_name[i]:
        indx = i
Billie1 = hundf.iloc[indx]
print(Billie1)
print(Billie2)

song_name              bad guy
song_artist      Billie Eilish
BPM                    131.926
energy                      43
loudness               -10.998
liveness                    10
valence                     56
song_duration              194
acousticness                33
speechiness                 38
Name: 786, dtype: object
song_name              bad guy
song_artist      Billie Eilish
BPM                    131.926
energy                      43
loudness               -10.998
liveness                    10
valence                     56
song_duration              194
acousticness                33
speechiness                 38
Name: 99368, dtype: object


In [62]:
hundf = hundf.round(2)
hundf = hundf.dropna()
hundf

,song_name,song_artist,BPM,energy,loudness,liveness,valence,song_duration,acousticness,speechiness
0,Sunrise,Norah Jones,153.43,30,-14.0,11,68,201,94,3
1,Black Night,Deep Purple,131.93,79,-11.0,17,81,207,17,7
2,Clint Eastwood,Gorillaz,164.17,69,-9.0,7,52,341,2,17
3,The Pretender,Foo Fighters,169.06,96,-4.0,3,37,269,0,4
4,Waitin' On A Sunny Day,Bruce Springsteen,103.59,82,-5.0,10,87,256,1,3
...,...,...,...,...,...,...,...,...,...,...
1989,Heartbreak Hotel,Elvis Presley,91.86,21,-12.0,11,72,128,84,7
1990,Hound Dog,Elvis Presley,171.02,76,-8.0,76,95,136,73,6
1991,Johnny B. Goode,Chuck Berry,164.17,80,-9.0,31,97,162,74,7
1992,Take Five,The Dave Brubeck Quartet,170.04,26,-13.0,7,60,324,54,4


In [63]:
moredf = moredf.round(2)
moredf = moredf.dropna()
moredf

,song_name,song_artist,BPM,energy,loudness,liveness,valence,song_duration,acousticness,speechiness
0,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",YG,203.93,34.87,-7.68,8.12,10.00,238.26,0.62,42.23
1,BAND DRUM (feat. A$AP Rocky),YG,159.01,57.30,-7.26,28.60,35.30,214.70,2.61,47.19
2,Radio Silence,R3HAB,114.97,74.38,-5.89,8.24,36.40,138.85,2.68,4.69
3,Lactose,Chris Cooq,123.00,59.56,-12.12,9.94,62.30,125.32,3.15,7.24
4,Same - Original mix,Chris Cooq,120.05,81.47,-10.28,3.32,91.00,123.96,0.00,6.83
...,...,...,...,...,...,...,...,...,...,...
130658,Come Back Home,Calum Scott,131.05,82.40,-5.17,9.91,27.10,190.45,0.73,3.34
130659,Enough for You,Saint Claire,94.04,25.61,-13.23,10.30,32.80,194.49,98.36,4.51
130660,Do It,Mike Stud,156.98,54.73,-8.35,9.97,52.80,139.13,35.36,21.27
130661,No Smoke,D Savage,120.05,52.46,-9.67,13.10,5.75,179.93,0.85,12.39


Now we want to merge our datasets in the event we want to do some manipulations later.

In [65]:
hundf['top2000']= True
moredf['top2000'] = False

jointdataf = [hundf,moredf]
jointdf = pd.concat(jointdataf)

In [67]:
jointdf

,song_name,song_artist,BPM,energy,loudness,liveness,valence,song_duration,acousticness,speechiness,top2000
0,Sunrise,Norah Jones,153.43,30.00,-14.00,11.00,68.00,201,94.00,3.00,True
1,Black Night,Deep Purple,131.93,79.00,-11.00,17.00,81.00,207,17.00,7.00,True
2,Clint Eastwood,Gorillaz,164.17,69.00,-9.00,7.00,52.00,341,2.00,17.00,True
3,The Pretender,Foo Fighters,169.06,96.00,-4.00,3.00,37.00,269,0.00,4.00,True
4,Waitin' On A Sunny Day,Bruce Springsteen,103.59,82.00,-5.00,10.00,87.00,256,1.00,3.00,True
...,...,...,...,...,...,...,...,...,...,...,...
130658,Come Back Home,Calum Scott,131.05,82.40,-5.17,9.91,27.10,190.45,0.73,3.34,False
130659,Enough for You,Saint Claire,94.04,25.61,-13.23,10.30,32.80,194.49,98.36,4.51,False
130660,Do It,Mike Stud,156.98,54.73,-8.35,9.97,52.80,139.13,35.36,21.27,False
130661,No Smoke,D Savage,120.05,52.46,-9.67,13.10,5.75,179.93,0.85,12.39,False
